# LEET 1298: Maximum Candies You Can Get from Boxes

Description omitted for copyright. Full problem statement can be found here:

<href>https://leetcode.com/problems/maximum-candies-you-can-get-from-boxes/



In [1]:
# imports
import time
from collections import deque

My solution (*solution1*) and a few of other solutions posted on the problem page for comparison. Perform a breadth-first search collecting keys and contained boxes starting with *inititalBoxes*. Return the sum of candies from the held boxes where the keys are also held. 

In [2]:
def solution1(status,candies,keys,containedBoxes,initialBoxes):

    # set of all visited boxes
    visited = set()
    # status_open. Instantiate with initital boxes
    status_open = set([i for i in range(len(status)) if status[i]==1])
    # set of boxes to visit. Instantiate with initial boxes if box status ==1 
    queue =set([initialBoxes[i] for i in range(len(initialBoxes)) if status[i]==1])
    # set of held boxes. Instantiate with initial boxes if box statuss is ==0
    held_boxes = set([initialBoxes[i] for i in range(len(initialBoxes)) if status[i]==0])
    # set of held keys
    held_keys = set()
    
    while queue:
        for v in queue:            
            # add current box to visited set
            visited.add(v)
            if v in status_open:
                # add v and any contained boxes to held boxes
                held_boxes.update(containedBoxes[v]+[v])
                # add any found keys to held keys
                held_keys.update(keys[v])
                
        # add intersection of held_keys and held_boxes to status_open
        status_open.update(held_boxes.intersection(held_keys))
        # recalc queue for next iter
        queue = status_open.intersection(held_boxes.difference(visited))
    
    return(sum([candies[i] for i in status_open if i in visited]))

def maxCandies1(status,candies,keys,containedBoxes,initialBoxes):
    # found here: https://leetcode.com/problems/maximum-candies-you-can-get-from-boxes/solutions/457653/python-simple-easy-to-understand-solution/
    """
    :type status: List[int]
    :type candies: List[int]
    :type keys: List[List[int]]
    :type containedBoxes: List[List[int]]
    :type initialBoxes: List[int]
    :rtype: int
    """

    # Inventory of boxes, keys and candies
    inv_boxes = initialBoxes.copy()
    # Reference to same object; i.e. repurpose initialBoxes rather than copying
    inv_keys = set()
    inv_candies = 0

		# Process boxes in our inventory
    while inv_boxes:

        took_stuff = False

        # If any boxes are opened/openable, take their contents (boxes, keys and 
        # candies) and throw away the box
        for box_id in inv_boxes:
            if status[box_id] == 1 or box_id in inv_keys:

                took_stuff = True
                # Add boxes to inventory
                inv_boxes.extend(containedBoxes[box_id])
                # Add keys to inventory
                for contained_key in keys[box_id]:
                    inv_keys.add(contained_key)
                # Add candies to inventory
                inv_candies += candies[box_id]
                # We're done with this box, so throw it away
                inv_boxes.remove(box_id)

        # If none of our boxes this round were opened/openable, then we can't proceed any further
        if not took_stuff:
            break
    return inv_candies

def maxCandies2(status,candies,keys,containedBoxes,initialBoxes):

    status = status.copy()
    q = deque(initialBoxes)
    visited = set()
    res = 0
    while q:
        itr, opened = len(q), False  # To detect cycle
        opened = False
        while (itr):
            itr -= 1
            v = q.popleft()
            if status[v]: # Open box, (key is available or is open)

                opened = True
                res += candies[v]
                visited.add(v)

                for x in keys[v]:
                    status[x] = 1

                for x in containedBoxes[v]:
                    if x not in visited:
                        q.append(x)

            elif v not in visited: # Open when key is available
                q.append(v)
        if not opened:
            return res  # Exit cycle detected
    return res

def maxCandies3(status,candies,keys,containedBoxes,initialBoxes):
    Q, total, keys_collected = deque(initialBoxes), 0, set()
    # BFS Implementation
    while Q:
        some_box_opened = False
        for _ in range(len(Q)):
            B = Q.popleft()
            if status[B] or B in keys_collected:
                # Box is opened
                some_box_opened = True
                total += candies[B]
                # Collecting Keys
                for key in keys[B]:
                    keys_collected.add(key)
                # Adding to Q such that these boxes can be opened later
                for box in containedBoxes[B]:
                    Q.append(box)
            else:
                Q.append(B)
        # If no box can be opened in the current iteration
        if not some_box_opened:
            break
    # Return total candies collected
    return total

def maxCandies4(status,candies,keys,containedBoxes,initialBoxes):
    status = status.copy()
    boxes = set(initialBoxes)
    bfs = [i for i in boxes if status[i]]
    for i in bfs:
        for j in containedBoxes[i]:
            boxes.add(j)
            if status[j]:
                bfs.append(j)
        for j in keys[i]:
            if status[j] == 0 and j in boxes:
                bfs.append(j)
            status[j] = 1
    return sum(candies[i] for i in bfs)

## Test Case 1

In [3]:
status = [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
candies = [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
keys = [[1, 19], [2, 8], [5, 3], [9], [10], [6], [7], [15], [], [16], [11], [12, 18], [], [], [13], [14], [17], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], []]
containedBoxes = ([1], [2, 3, 4], [5, 6, 7], [8, 9], [10, 11, 12], [], [], [13, 14, 15], [], [16, 17], [18], [], [19], [], [], [], [], [], [], [], [21, 22, 23, 24], [25, 26], [27, 28], [29, 30], [], [34, 35], [36], [37], [], [38], [39], [], [], [], [], [], [], [], [], [], [41, 42, 43], [44, 45], [46], [47], [48], [49], [], [], [], [])
initialBoxes = [0]

print(f'Max candies, solution 1: {solution1(status,candies,keys,containedBoxes,initialBoxes)}')
print(f'Max candies, maxCandies1: {maxCandies1(status,candies,keys,containedBoxes,initialBoxes)}')
print(f'Max candies, maxCandies2: {maxCandies2(status,candies,keys,containedBoxes,initialBoxes)}')
print(f'Max candies, maxCandies3: {maxCandies3(status,candies,keys,containedBoxes,initialBoxes)}')
print(f'Max candies, maxCandies4: {maxCandies4(status,candies,keys,containedBoxes,initialBoxes)}')

n_iter = 10**4

t0 = time.time()
for _ in range(n_iter):
    solution1(status,candies,keys,containedBoxes,initialBoxes)
t1 = time.time()
print(f'\nSolution1: {t1-t0}')

t0 = time.time()
for _ in range(n_iter):
    maxCandies1(status,candies,keys,containedBoxes,initialBoxes)
t1 = time.time()
print(f'maxCandies1: {t1-t0}')

t0 = time.time()
for _ in range(n_iter):
    maxCandies2(status,candies,keys,containedBoxes,initialBoxes)
t1 = time.time()
print(f'maxCandies2: {t1-t0}')

t0 = time.time()
for _ in range(n_iter):
    maxCandies3(status,candies,keys,containedBoxes,initialBoxes)
t1 = time.time()
print(f'maxCandies3: {t1-t0}')

t0 = time.time()
for _ in range(n_iter):
    maxCandies4(status,candies,keys,containedBoxes,initialBoxes)
t1 = time.time()
print(f'maxCandies4: {t1-t0}')

Max candies, solution 1: 14
Max candies, maxCandies1: 14
Max candies, maxCandies2: 14
Max candies, maxCandies3: 14
Max candies, maxCandies4: 14

Solution1: 0.3723485469818115
maxCandies1: 0.19349002838134766
maxCandies2: 0.1504514217376709
maxCandies3: 0.17993617057800293
maxCandies4: 0.11277651786804199


## Test Case 2

In [4]:
status= [1, 0, 0, 0, 0, 0, 0, 0, 1, 0]
candies= [20, 4, 17, 6, 9, 2, 19, 14, 8, 6]
keys= [[9, 1, 0, 2], [], [], [], [3, 4], [6, 5], [7], [], [], [8]]
containedBoxes= [[3, 6], [], [], [], [1], [], [0], [9, 4], [2], [5, 8, 7]]
initialBoxes= [0]

print(f'Max candies, solution 1: {solution1(status,candies,keys,containedBoxes,initialBoxes)}')
print(f'Max candies, maxCandies1: {maxCandies1(status,candies,keys,containedBoxes,initialBoxes)}')
print(f'Max candies, maxCandies2: {maxCandies2(status,candies,keys,containedBoxes,initialBoxes)}')
print(f'Max candies, maxCandies3: {maxCandies3(status,candies,keys,containedBoxes,initialBoxes)}')
print(f'Max candies, maxCandies4: {maxCandies4(status,candies,keys,containedBoxes,initialBoxes)}')

n_iter = 10**4

t0 = time.time()
for _ in range(n_iter):
    solution1(status,candies,keys,containedBoxes,initialBoxes)
t1 = time.time()
print(f'\nSolution1: {t1-t0}')

t0 = time.time()
for _ in range(n_iter):
    maxCandies1(status,candies,keys,containedBoxes,initialBoxes)
t1 = time.time()
print(f'maxCandies1: {t1-t0}')

t0 = time.time()
for _ in range(n_iter):
    maxCandies2(status,candies,keys,containedBoxes,initialBoxes)
t1 = time.time()
print(f'maxCandies2: {t1-t0}')

t0 = time.time()
for _ in range(n_iter):
    maxCandies3(status,candies,keys,containedBoxes,initialBoxes)
t1 = time.time()
print(f'maxCandies3: {t1-t0}')

t0 = time.time()
for _ in range(n_iter):
    maxCandies4(status,candies,keys,containedBoxes,initialBoxes)
t1 = time.time()
print(f'maxCandies4: {t1-t0}')

Max candies, solution 1: 20
Max candies, maxCandies1: 20
Max candies, maxCandies2: 20
Max candies, maxCandies3: 20
Max candies, maxCandies4: 20

Solution1: 0.08392763137817383
maxCandies1: 0.01662158966064453
maxCandies2: 0.02399921417236328
maxCandies3: 0.02369379997253418
maxCandies4: 0.028187274932861328


## Test Case 3

In [5]:
status = [1,0,1,0]
candies = [7,5,4,100]
keys = [[],[],[1],[]]
containedBoxes = [[1,2],[3],[],[]]
initialBoxes = [0]

print(f'Max candies, solution 1: {solution1(status,candies,keys,containedBoxes,initialBoxes)}')
print(f'Max candies, maxCandies1: {maxCandies1(status,candies,keys,containedBoxes,initialBoxes)}')
print(f'Max candies, maxCandies2: {maxCandies2(status,candies,keys,containedBoxes,initialBoxes)}')
print(f'Max candies, maxCandies3: {maxCandies3(status,candies,keys,containedBoxes,initialBoxes)}')
print(f'Max candies, maxCandies4: {maxCandies4(status,candies,keys,containedBoxes,initialBoxes)}')

n_iter = 10**4

t0 = time.time()
for _ in range(n_iter):
    solution1(status,candies,keys,containedBoxes,initialBoxes)
t1 = time.time()
print(f'\nSolution1: {t1-t0}')

t0 = time.time()
for _ in range(n_iter):
    maxCandies1(status,candies,keys,containedBoxes,initialBoxes)
t1 = time.time()
print(f'maxCandies1: {t1-t0}')

t0 = time.time()
for _ in range(n_iter):
    maxCandies2(status,candies,keys,containedBoxes,initialBoxes)
t1 = time.time()
print(f'maxCandies2: {t1-t0}')

t0 = time.time()
for _ in range(n_iter):
    maxCandies3(status,candies,keys,containedBoxes,initialBoxes)
t1 = time.time()
print(f'maxCandies3: {t1-t0}')

t0 = time.time()
for _ in range(n_iter):
    maxCandies4(status,candies,keys,containedBoxes,initialBoxes)
t1 = time.time()
print(f'maxCandies4: {t1-t0}')

Max candies, solution 1: 16
Max candies, maxCandies1: 16
Max candies, maxCandies2: 16
Max candies, maxCandies3: 16
Max candies, maxCandies4: 16

Solution1: 0.10172581672668457
maxCandies1: 0.027217626571655273
maxCandies2: 0.03782939910888672
maxCandies3: 0.03725290298461914
maxCandies4: 0.03491830825805664


## Test Case 4

In [6]:
status = [1,0,0,0,0,0]
candies = [1,1,1,1,1,1]
keys = [[1,2,3,4,5],[],[],[],[],[]]
containedBoxes = [[1,2,3,4,5],[],[],[],[],[]]
initialBoxes = [0]

print(f'Max candies, solution 1: {solution1(status,candies,keys,containedBoxes,initialBoxes)}')
print(f'Max candies, maxCandies1: {maxCandies1(status,candies,keys,containedBoxes,initialBoxes)}')
print(f'Max candies, maxCandies2: {maxCandies2(status,candies,keys,containedBoxes,initialBoxes)}')
print(f'Max candies, maxCandies3: {maxCandies3(status,candies,keys,containedBoxes,initialBoxes)}')
print(f'Max candies, maxCandies4: {maxCandies4(status,candies,keys,containedBoxes,initialBoxes)}')

n_iter = 10**4

t0 = time.time()
for _ in range(n_iter):
    solution1(status,candies,keys,containedBoxes,initialBoxes)
t1 = time.time()
print(f'\nSolution1: {t1-t0}')

t0 = time.time()
for _ in range(n_iter):
    maxCandies1(status,candies,keys,containedBoxes,initialBoxes)
t1 = time.time()
print(f'maxCandies1: {t1-t0}')

t0 = time.time()
for _ in range(n_iter):
    maxCandies2(status,candies,keys,containedBoxes,initialBoxes)
t1 = time.time()
print(f'maxCandies2: {t1-t0}')

t0 = time.time()
for _ in range(n_iter):
    maxCandies3(status,candies,keys,containedBoxes,initialBoxes)
t1 = time.time()
print(f'maxCandies3: {t1-t0}')

t0 = time.time()
for _ in range(n_iter):
    maxCandies4(status,candies,keys,containedBoxes,initialBoxes)
t1 = time.time()
print(f'maxCandies4: {t1-t0}')

Max candies, solution 1: 6
Max candies, maxCandies1: 6
Max candies, maxCandies2: 6
Max candies, maxCandies3: 6
Max candies, maxCandies4: 6

Solution1: 0.14493823051452637
maxCandies1: 0.06859803199768066
maxCandies2: 0.10706925392150879
maxCandies3: 0.051802873611450195
maxCandies4: 0.0592803955078125
